Directory Generation

In [ ]:
!gdown https://drive.google.com/uc?id=15JsVCUxYsSGFRwGhqre0zLpSIg3-jLvO #Downloading zip file from google drive
zip_path = '/content/Cat_Classifier_Images_70-30-20200828T022923Z-001.zip' #Getting the path 
!unzip -q Cat_Classifier_Images_70-30-20200829T163124Z-001.zip #Unzipping the folder
!rm Cat_Classifier_Images_70-30-20200829T163124Z-001.zip #Removing the zip folder
import os #Imported required libraries for this step
import pdb
import cv2
import operator
from PIL import Image
os.mkdir("Training-70-30") #making the new Training and testing folders
os.mkdir("Testing-70-30")
dirs = ["Domino", "Stormy"] #Making the names for the subdirec
pardirs = ["/content/Training-70-30", "/content/Testing-70-30"]
for pardir in pardirs:
  for dir in dirs:
    path = os.path.join(pardir, dir)
    os.mkdir(path)
datadir1 = "/content/Cat_Classifier_Images_70-30 /Train/Domino"
datadir2 = "/content/Cat_Classifier_Images_70-30 /Train/Stormy"
datadir3 = "/content/Cat_Classifier_Images_70-30 /Test/Domino"
datadir4 = "/content/Cat_Classifier_Images_70-30 /Test/Stormy"
filelist1 = sorted(os.listdir(datadir1), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist2 = sorted(os.listdir(datadir2), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist3 = sorted(os.listdir(datadir3), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist4 = sorted(os.listdir(datadir4), key = lambda fname: int(fname.split("_")[0][-4:]))
datadirs1 = [filelist1, filelist2, filelist3, filelist4]
inc = 0
for filelist in datadirs1:
  i = 0
  for fil in filelist:
    if inc == 0:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Domino/" + fil
      i += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (96, 128))
      cv2.imwrite('/content/Training-70-30/Domino/DominoTR%02i.jpg' %i, imgResized)
    elif inc == 1:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Stormy/" + fil
      i += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (96, 128))
      cv2.imwrite('/content/Training-70-30/Stormy/StormyTR%02i.jpg' %i, imgResized)
    elif inc == 2:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Domino/" + fil
      i += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (96, 128))
      cv2.imwrite('/content/Testing-70-30/Domino/DominoTS%02i.jpg' %i, imgResized)
    else:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Stormy/" + fil
      i += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (96, 128))
      cv2.imwrite('/content/Testing-70-30/Stormy/StormyTS%02i.jpg' %i, imgResized)
  inc += 1

Making, training, and validating my classifier

In [ ]:
import numpy as np #Importing needed libraries
from tensorflow import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing import image
from keras.regularizers import l1, l2, l1_l2
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
seed = 1
np.random.seed(seed)
random.set_seed(seed)
classifier = Sequential() 
classifier.add(Conv2D(64, (3, 3), input_shape = (64, 48, 3), activation = 'relu')) 
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 48, 3), activation = 'relu'))
classifier.add(MaxPool2D(2, 2)) #2 MaxPools for lessening the amount of trained parameters in the network
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 48, 3), activation = 'relu'))
classifier.add(MaxPool2D(2, 2))
classifier.add(Flatten())
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dropout(rate = 0.1)) 
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])
epochs = 2000
train_datagen = ImageDataGenerator(rescale = 1./255.0, shear_range = 0.3, zoom_range = 0.1, horizontal_flip = True, vertical_flip = True, width_shift_range = 0.2, height_shift_range = 0.2) #Set up image directories
test_datagen = ImageDataGenerator(rescale = 1./255.0, shear_range = 0.3, zoom_range = 0.1, horizontal_flip = True, vertical_flip = True, width_shift_range = 0.2, height_shift_range = 0.2)
train_set = train_datagen.flow_from_directory('/content/Training-70-30', target_size = (64, 48), batch_size = 10, class_mode = 'binary', seed = seed) #Load in images
test_set = test_datagen.flow_from_directory('/content/Testing-70-30', target_size = (64, 48), batch_size = 5, class_mode = 'binary', seed = seed)
history = classifier.fit(train_set, steps_per_epoch = 14, epochs = epochs, validation_data = test_set, validation_steps = 12, shuffle = True, verbose = False)
epochs1 = [i for i in range(epochs)]
plt.scatter(epochs1, history.history['val_accuracy'])
plt.ylabel("Validation Accuracy")
plt.xlabel("Epochs")
plt.show()
plt.scatter(epochs1, history.history["accuracy"])
plt.xlabel("Epochs")
plt.ylabel("Training Accuracy")
plt.show()

Testing the classifier

In [ ]:
from keras.preprocessing import image
import cv2
exp_img = cv2.imread('/content/Testing-70-30/Stormy/StormyTS06.jpg')
exp2_img = cv2.resize(exp_img, (64, 48))
exp2_img = image.img_to_array(exp2_img)
exp2_img = np.expand_dims(exp2_img, axis = 0)
result = classifier.predict(exp2_img)
if result[0][0] == 1:
  print ('This image is Domino')
else:
  print ("This image is Stormy")